* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
* 별점을 시청횟수로 해석해서 생각하겠습니다.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import implicit
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


# 데이터 준비와 전처리

In [2]:
rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding='ISO-8859-1')
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남김.
ratings = ratings[ratings['ratings'] >= 3]
filtered_data_size = len(ratings)

print(f'original_data_size : {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size : 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 교체.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 별점을 시청횟수로 바꿧으니, 별점을 등록한 timestamp는 제거
ratings = ratings[['user_id', 'movie_id', 'counts']]
ratings

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [8]:
movies[movies['movie_id'] == 1193]['title']

1176    One Flew Over the Cuckoo's Nest (1975)
Name: title, dtype: object

In [9]:
# user 1이 본 영화들
for i in ratings[ratings['user_id'] == 1]['movie_id'].values:
    print(movies[movies['movie_id'] == i]['title'].values[0])

One Flew Over the Cuckoo's Nest (1975)
James and the Giant Peach (1996)
My Fair Lady (1964)
Erin Brockovich (2000)
Bug's Life, A (1998)
Princess Bride, The (1987)
Ben-Hur (1959)
Christmas Story, A (1983)
Snow White and the Seven Dwarfs (1937)
Wizard of Oz, The (1939)
Beauty and the Beast (1991)
Gigi (1958)
Miracle on 34th Street (1947)
Ferris Bueller's Day Off (1986)
Sound of Music, The (1965)
Airplane! (1980)
Tarzan (1999)
Bambi (1942)
Awakenings (1990)
Big (1988)
Pleasantville (1998)
Wallace & Gromit: The Best of Aardman Animation (1996)
Back to the Future (1985)
Schindler's List (1993)
Meet Joe Black (1998)
Pocahontas (1995)
E.T. the Extra-Terrestrial (1982)
Titanic (1997)
Ponette (1996)
Close Shave, A (1995)
Antz (1998)
Girl, Interrupted (1999)
Hercules (1997)
Aladdin (1992)
Mulan (1998)
Hunchback of Notre Dame, The (1996)
Last Days of Disco, The (1998)
Cinderella (1950)
Sixth Sense, The (1999)
Apollo 13 (1995)
Toy Story (1995)
Rain Man (1988)
Driving Miss Daisy (1989)
Run Lola Run

# 분석해 봅시다.

* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [10]:
# ratings 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [11]:
# ratings 유니크한 사용자 개수
ratings['user_id'].nunique()

6039

In [12]:
# rating에는 movie_id 만 있으니 title을 위해 합침.
user_df = pd.merge(ratings, movies, on='movie_id')

In [13]:
# 가장 인기 있는 영화 30개
movie_rating = user_df.groupby('title')['title'].count()
movie_rating.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [14]:
my_favorite = ['Phantom of the Opera, The (1943)', 'Modern Times (1936)', 'Gone with the Wind (1939)', 'Roman Holiday (1953)', 'Ben-Hur (1959)']
my_favorite_id = [3936, 3462, 920, 916, 1287]
my_favorite_gr = ['Drama|Thriller', 'Comedy', 'Drama|Romance|War', 'Comedy|Romance', 'Action|Adventure|Drama']

# user_id 맨 마지막에 추가를 위해 6040 다음인 6041에 본인 정보 넣기.
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id':my_favorite_id, 'counts': [5]*5, 'title': my_favorite, 'genre': my_favorite_gr})

if not user_df.isin({'user_id':[6041]})['user_id'].any():
    user_df = user_df.append(my_playlist)
    
#ratings.reset_index(drop=True)
user_df.tail(10)

,user_id,movie_id,counts,title,genre
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,Identification of a Woman (Identificazione di ...,Drama
0,6041,3936,5,"Phantom of the Opera, The (1943)",Drama|Thriller
1,6041,3462,5,Modern Times (1936),Comedy
2,6041,920,5,Gone with the Wind (1939),Drama|Romance|War
3,6041,916,5,Roman Holiday (1953),Comedy|Romance
4,6041,1287,5,Ben-Hur (1959),Action|Adventure|Drama


### 모델  활용을 위한 indexing

In [15]:
user_unique = user_df['user_id'].unique()
movies_unique = user_df['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movies_to_idx = {v:k for k,v in enumerate(movies_unique)}

In [16]:
# 인덱싱 확인
print(user_to_idx[6041])
print(movies_to_idx['Phantom of the Opera, The (1943)'])

6039
2106


In [17]:
temp_user_data = user_df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == (len(user_df)):
    print('user_id columns indexing OK!!')
    user_df['user_id'] = temp_user_data
else:
    print('user_id column indexing Fail!!')

temp_movies_data = user_df['title'].map(movies_to_idx.get).dropna()
if len(temp_movies_data) == len(user_df):
    print('title columns indexing OK!!')
    user_df['title'] = temp_movies_data
else:
    print('title column indexing Fail!!')

user_df

user_id columns indexing OK!!
title columns indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,1193,5,0,Drama
1,1,1193,5,0,Drama
2,2,1193,4,0,Drama
3,3,1193,4,0,Drama
4,4,1193,5,0,Drama
...,...,...,...,...,...
0,6039,3936,5,2106,Drama|Thriller
1,6039,3462,5,1329,Comedy
2,6039,920,5,143,Drama|Romance|War
3,6039,916,5,588,Comedy|Romance


# CSR matrix를 직접 만들어 봅시다.

In [18]:
num_user = user_df['user_id'].nunique()
num_movies = user_df['title'].nunique()

csr_data = csr_matrix((user_df.counts, (user_df.user_id, user_df.title)), shape=(num_user, num_movies))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [19]:
# implicit 라이브러리에서 권장하는 부분

os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
# factors와 iterations를 늘릴 수록 학습 데이터를 잘 학습하게 되지만 과적합 우려가 있습니다.
als_model = AlternatingLeastSquares(factors=1200, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 형태의 matrix를 받기 때문에 Transpose)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [23]:
# 본인 user_id 6041과 선호 영화 movie_id 3936 의 벡터 확인
user_test, phantom = user_to_idx[6041], movies_to_idx['Phantom of the Opera, The (1943)']
user_vector, phantom_vector = als_model.user_factors[user_test], als_model.item_factors[phantom]

In [24]:
user_vector

array([ 0.0477744 , -0.00278695,  0.3976754 , ...,  0.15339395,
       -0.2905468 ,  0.13049494], dtype=float32)

In [25]:
phantom_vector

array([0.00243249, 0.00866538, 0.00824687, ..., 0.00510648, 0.005526  ,
       0.01159785], dtype=float32)

In [26]:
# 두 값의 내적
np.dot(user_vector, phantom_vector)

0.705632

선호 영화가 0.705 이라는 수치가 출력되었습니다. 어느정도 연관

# 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [27]:
# 선호영화 modern times
modern = movies_to_idx['Modern Times (1936)']
modern_vector = als_model.item_factors[modern]
np.dot(user_vector, modern_vector)

0.84188956

In [28]:
# 비선호 영화 toy story
toy = movies_to_idx['Toy Story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(user_vector, toy_vector)

-0.008860809

# 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [29]:
idx_to_movies = {v:k for k,v in movies_to_idx.items()}

In [30]:
def get_similar_movie(movie_title: str):
    # movie title을 id값으로 변환
    movie_id = movies_to_idx[movie_title]
    # movie id로 영화 추천받기
    similar_movie = als_model.similar_items(movie_id, N=15)
    # 추천 받은 movie id를 title로 변환
    similar_movie = [idx_to_movies[i[0]] for i in similar_movie]
    return similar_movie

In [31]:
get_similar_movie('Roman Holiday (1953)')

['Roman Holiday (1953)',
 'Coming Apart (1969)',
 'Allnighter, The (1987)',
 'Separation, The (La Séparation) (1994)',
 'Bay of Blood (Reazione a catena) (1971)',
 'Slappy and the Stinkers (1998)',
 'Country Life (1994)',
 'Nemesis 2: Nebula (1995)',
 'Illtown (1996)',
 'Paris Was a Woman (1995)',
 'Penitentiary II (1982)',
 'Ring, The (1927)',
 'Spiders, The (Die Spinnen, 1. Teil: Der Goldene See) (1919)',
 'I Got the Hook Up (1998)',
 'Male and Female (1919)']

In [32]:
get_similar_movie('Modern Times (1936)')

['Modern Times (1936)',
 'Ulysses (Ulisse) (1954)',
 'Gay Deceivers, The (1969)',
 'Coming Apart (1969)',
 'Chain of Fools (2000)',
 'Mamma Roma (1962)',
 'Slappy and the Stinkers (1998)',
 'Boys (1996)',
 'Last Time I Saw Paris, The (1954)',
 'Back Stage (2000)',
 'Telling You (1998)',
 'Intimate Relations (1996)',
 '24-hour Woman (1998)',
 'Alley Cats, The (1968)',
 'Macao (1952)']

# 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [33]:
user = user_to_idx[6041]
# recommend 에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1711, 0.18017456),
 (1930, 0.15042284),
 (1328, 0.13500012),
 (989, 0.1298891),
 (858, 0.11970505),
 (1655, 0.11884551),
 (1347, 0.11497605),
 (612, 0.10857285),
 (592, 0.10354547),
 (1530, 0.09668364),
 (718, 0.09586421),
 (925, 0.09353377),
 (1448, 0.09107827),
 (541, 0.08982909),
 (1704, 0.08967006),
 (585, 0.08928774),
 (1830, 0.08875323),
 (660, 0.08523773),
 (536, 0.083261296),
 (503, 0.08314226)]

In [34]:
[idx_to_movies[i[0]] for i in movie_recommended]

['City Lights (1931)',
 'Bank Dick, The (1940)',
 'Great Dictator, The (1940)',
 'Creature From the Black Lagoon, The (1954)',
 'Invisible Man, The (1933)',
 'Voyage to the Bottom of the Sea (1961)',
 'Born Yesterday (1950)',
 'Corrina, Corrina (1994)',
 'Gold Rush, The (1925)',
 'Crocodile Dundee II (1988)',
 'Abbott and Costello Meet Frankenstein (1948)',
 'Fantastic Voyage (1966)',
 'Next Stop, Wonderland (1998)',
 'All Quiet on the Western Front (1930)',
 'Karate Kid, Part II, The (1986)',
 'Sabrina (1954)',
 'Return of the Fly (1959)',
 'Philadelphia Story, The (1940)',
 'Jungle Book, The (1967)',
 'Duck Soup (1933)']

In [35]:
# 추천받은것에 대한 기여도 확인
excalibur = movies_to_idx['Great Dictator, The (1940)']
explain = als_model.explain(user, csr_data, itemid=excalibur)

In [36]:
[(idx_to_movies[i[0]], i[1]) for i in explain[1]]

[('Modern Times (1936)', 0.13225692687116597),
 ('Phantom of the Opera, The (1943)', 0.0071912590829049244),
 ('Roman Holiday (1953)', 0.0027301092391267007),
 ('Ben-Hur (1959)', -0.003298385642182965),
 ('Gone with the Wind (1939)', -0.006539682297467923)]

In [37]:
movies[movies['title'] == 'City Lights (1931)']

,movie_id,title,genre
3238,3307,City Lights (1931),Comedy|Drama|Romance


In [38]:
movies[movies['title'] == 'Modern Times (1936)']

,movie_id,title,genre
3393,3462,Modern Times (1936),Comedy


In [39]:
movies[movies['title'] == 'Phantom of the Opera, The (1943)']

,movie_id,title,genre
3866,3936,"Phantom of the Opera, The (1943)",Drama|Thriller


추천 기여도 상위 2개 항목을 출력해보니, 장르에 Comedy 와 Drama가 추천도에 기여한 것같습니다.